In [1]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset

mlonazure_ws = Workspace.from_config()

mlonazure_ds = mlonazure_ws.get_default_datastore()

print('Workspace Name: ' + mlonazure_ws.name, 
      'Resource Group: ' + mlonazure_ws.resource_group,
      'Default Storage Account Name: ' + mlonazure_ds.account_name,
      'AzureML Core Version: ' + azureml.core.VERSION,
      sep = '\n')

Workspace Name: mlonazure-ws
Resource Group: mlonazure-rg
Default Storage Account Name: mlonazurews7669880942
AzureML Core Version: 1.8.0


In [2]:
from azureml.core.compute import AmlCompute

compute_target = AmlCompute(mlonazure_ws, 'amlcluster2c-4gb')

print('Compute Name:', compute_target.name)

Compute Name: amlcluster2c-4gb


In [3]:
#Note that you need to run ../00. Get Data from OpenDatasets.ipynb to get and register this dataset.
datasetName_train = 'OJSales_Train_Subset'
datasetName_validate = 'OJSales_Validate_Subset'

data_train_ds = Dataset.get_by_name(mlonazure_ws,datasetName_train,version='latest')
data_validate_ds = Dataset.get_by_name(mlonazure_ws,datasetName_validate,version='latest')

In [4]:
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig

import logging

target_column_name = 'Quantity'
dateColumn = 'WeekStarting'

training_data = data_train_ds

automl_config = AutoMLConfig(task='forecasting',
                             path = './project',
                             debug_log='automl_debuglog.log',
                             primary_metric='r2_score',
                             iteration_timeout_minutes = 120,
                             experiment_timeout_hours=10,
                             featurization='auto',
                             max_concurrent_iterations=20, #100 is the max but its one per node so max should be set to the max nodes.
                             max_cores_per_iteration=-1,
                             enable_dnn=False,
                             enable_early_stopping=True,
                             n_cross_validations=3,
                             verbosity = logging.INFO,                             
                             compute_target=compute_target,
                             training_data=training_data,
                             label_column_name=target_column_name,
                             time_column_name = dateColumn,
                             max_horizon = 6,
                             grain_column_names = ['Store', 'Brand'],
                             #drop_column_names = ['',''],
                             model_explainability=True)

In [5]:
from azureml.pipeline.core import PipelineData, TrainingOutput, PipelineDataset
from azureml.pipeline.steps import AutoMLStep

metrics_data = PipelineData(name='metrics_data',
                           datastore=mlonazure_ds,
                           pipeline_output_name='metrics_output_name',
                           training_output=TrainingOutput(type='Metrics'))

model_data_bestmodel = PipelineData(name='model_data_bestmodel',
                           datastore=mlonazure_ds,
                           pipeline_output_name='best_model_output_name',
                           training_output=TrainingOutput(type='Model'))

model_data_normalizedrmse = PipelineData(name='model_data_normalizedrmse',
                           datastore=mlonazure_ds,
                           pipeline_output_name='nrmse_model_output_name',
                           training_output=TrainingOutput(type='Model',metric='normalized_root_mean_squared_error'))

automl_step = AutoMLStep(name='TrainOJSalesData_AutoML',
                        automl_config=automl_config,
                        inputs=None,
                        outputs=[metrics_data, model_data_bestmodel,model_data_normalizedrmse],
                        allow_reuse=False,
                        version=None,
                        passthru_automl_config=True)

In [6]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

source_directory = 'Scripts'
model_name = 'OJForecastingModel'
model_description = 'Built using AutoML'
#registeredModelInput = InputPortBinding("bestmodel", bind_object=model_data_bestmodel)

pyscriptStepRegisterModel = PythonScriptStep(name="pystepRegisterModel",
                                             script_name="registermodel.py",
                                             arguments=["--model_path", model_data_bestmodel, 
                                                        "--model_name", model_name, 
                                                        "--model_description",model_description],
                                             inputs=[model_data_bestmodel],
                                             compute_target=compute_target, 
                                             source_directory=source_directory,
                                             allow_reuse=False)

print("pyscriptStepRegisterModel Script Step Created")

pyscriptStepRegisterModel Script Step Created


In [7]:
from azureml.pipeline.core import Pipeline

amlpipeline = Pipeline(workspace=mlonazure_ws, 
         steps=[automl_step,pyscriptStepRegisterModel],
         description= 'ForecastOJSales Training Pipeline! July 2020')

In [ ]:
from azureml.core import Experiment 

experiment_name = 'OJ_Sales_Quantity_Prediction_Pipeline'
experiment = Experiment(workspace = mlonazure_ws, name = experiment_name)

experiment.tag('Author', 'mlonazure')
experiment.tag('Department', 'Finance')
experiment.tag('Executor', 'Manual')

exp_pipelinerun = experiment.submit(amlpipeline)

exp_pipelinerun.wait_for_completion()

In [8]:
amlpipeline.publish(
        name='OJSalesTrainingPipeline July', 
        description='AML Pipeline for automation July', 
        version=None, 
        continue_on_step_failure=False)

Created step TrainOJSalesData_AutoML [1d646e61][3098c279-c519-4169-8148-940ac500ddd5], (This step will run and generate new outputs)
Created step pystepRegisterModel [b81edef3][31dbdf24-c178-4c64-ac52-5c98be8a6d02], (This step will run and generate new outputs)


Name,Id,Status,Endpoint
OJSalesTrainingPipeline July,77726f6d-b73e-43a9-9540-a5a5511c51ff,Active,REST Endpoint


In [9]:
from azureml.pipeline.core import PipelineEndpoint

pipelineEndPoint = PipelineEndpoint.publish(workspace=mlonazure_ws, name="OJSales Prediction EndPoint",
                                            pipeline=amlpipeline, description="Pipeline EndPoint for OJSales Prediction")
pipelineEndPoint

#Note that once you have published a pipeline, you can add other pipelines to it using, pipelineEndPoint.add(publishedPipeline)

PipelineEndpoint(Name: OJSales Prediction EndPoint,
Id: e54c2487-cd98-4aff-807e-c952d7280368,
Description: Pipeline EndPoint for OJSales Prediction,
Pipelines: [('0', Pipeline(Name: OJSales Prediction EndPoint-07-09-2020-13-16-Pipeline,
Id: 6ab9eb5b-ac3b-4df8-9149-244663e184a6,
Status: Active,
Endpoint: https://eastus2.api.azureml.ms/pipelines/v1.0/subscriptions/286d2a8f-e90b-4e3f-b4d7-08971a2ab3fe/resourceGroups/mlonazure-rg/providers/Microsoft.MachineLearningServices/workspaces/mlonazure-ws/PipelineRuns/PipelineSubmit/6ab9eb5b-ac3b-4df8-9149-244663e184a6))])

In [10]:
from azureml.core import Experiment 

experiment_name = 'OJ_Sales_Quantity_Prediction_Pipeline'
experiment = Experiment(workspace = mlonazure_ws, name = experiment_name)

experiment.tag('Author', 'mlonazure')
experiment.tag('Department', 'Finance')
experiment.tag('Executor', 'Manual with EndPoint')

exp_pipelinerun = experiment.submit(pipelineEndPoint)

exp_pipelinerun.wait_for_completion()

PipelineRunId: 99ec2d4f-681a-4e4b-bbd5-89aa0d5b1e8f
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/OJ_Sales_Quantity_Prediction_Pipeline/runs/99ec2d4f-681a-4e4b-bbd5-89aa0d5b1e8f?wsid=/subscriptions/286d2a8f-e90b-4e3f-b4d7-08971a2ab3fe/resourcegroups/mlonazure-rg/workspaces/mlonazure-ws
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 89e8bdb9-2b57-44a9-a620-134b0c81ec2c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/OJ_Sales_Quantity_Prediction_Pipeline/runs/89e8bdb9-2b57-44a9-a620-134b0c81ec2c?wsid=/subscriptions/286d2a8f-e90b-4e3f-b4d7-08971a2ab3fe/resourcegroups/mlonazure-rg/workspaces/mlonazure-ws
StepRun( TrainOJSalesData_AutoML ) Status: NotStarted
StepRun( TrainOJSalesData_AutoML ) Status: Running

StepRun(TrainOJSalesData_AutoML) Execution Summary
StepRun( TrainOJSalesData_AutoML ) Status: Finished
{'runId': '89e8bdb9-2b57-44a9-a620-134b0c81ec2c', 'target': 'amlcluster2c-4gb', 'status': 'Completed', 'star

'Finished'